In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast

In [2]:
# size_width_rugs_runners
# size_length_rugs_runners
# size_diameter_rugs_runners

# parameters
customer_id = '5'
formatted_attribute = 'size_width_rugs_runners'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

buckets = """Rugs & Mats B"""


# regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|()"
regex_pattern = r"(?i)(\s[^t]\d+\-Inch x \d+\-Inch)|(\d+\\\" x \d*\\\")"


value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'

In [3]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [4]:
df = query_from_file(file_name='query/bbby.sql', params=params)
df

attribute       value    customers_name external_id  \
0      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005824   
1      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005831   
2      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005855   
3      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005893   
4      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005909   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
0      Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Dark Brown (Set of 2)   
1           Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Camel (Set of 2)   
2        Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Charcoal (Set of 2)   
3            Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Grey (Set of 2)   
4      Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Blue Stone (Set of 2)   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution  
0          rules  
1          rules  
2          rules  
3          rules  
4          rules  
...          ...  
56956   standard  
56957   standard  
56958   standard  
56959   standard  
56960   standard  

[56961 rows x 6 columns]

In [5]:
# Find a list of values to iterate
df_strategy = query_from_file(file_name='./asset/easy.sql', params=params) #create query
df_strategy
list_values=df_strategy['value'].to_list()
list_values

['0" - 11.9"',
 '108" or Greater',
 '12" - 23.9"',
 '24" - 35.9"',
 '36" - 47.9"',
 '48" - 59.9"',
 '60" - 71.9"',
 '72" - 83.9"',
 '84" - 95.9"',
 '96" - 107.9"',
 'n/a']

In [6]:
metric=""
range_string =""""

 Greater than 108
 12 - 23.9
 24 - 35.9
 36 - 47.9
 48 - 59.9
 60 - 71.9
 72 - 83.9
 84 - 95.9
 96 - 107.9
 0 - 11.9
 """

In [7]:
# kwp=KeywordProcessor()
# kwp.add_keywords_from_list(list_valuez)
# print(kwp.add_keywords_from_list(list_values))
## Extract Key Words
# valuez=x['Max'].apply(lambda x: kwp.extract_keywords(x))
# valuez[0:50]

In [8]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [9]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

attribute       value    customers_name external_id  \
0      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005824   
1      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005831   
2      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005855   
3      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005893   
4      size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    45005909   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
0      Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Dark Brown (Set of 2)   
1           Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Camel (Set of 2)   
2        Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Charcoal (Set of 2)   
3            Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Grey (Set of 2)   
4      Weather Guard™ Broken Brick 30-Inch x 8.5-Inch Stair Treads in Blue Stone (Set of 2)   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution        matches  
0          rules             []  
1          rules             []  
2          rules             []  
3          rules             []  
4          rules             []  
...          ...            ...  
56956   standard  [22\" x 35\"]  
56957   standard  [22\" x 35\"]  
56958   standard  [22\" x 35\"]  
56959   standard  [22\" x 35\"]  
56960   standard  [22\" x 35\"]  

[56961 rows x 7 columns]

In [10]:
# df['matches'] = df['long_desc'].apply(lambda x: kwp.extract_keywords(x))
# df

In [11]:
x=df[df['matches'].astype(str)!='[]']
x

attribute       value    customers_name external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    60715197   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550184   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550603   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63561005   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution              matches  
44         rules  [ 35-Inch x 9-Inch]  
45      standard          [5\" x 8\"]  
46      standard        [30\" x 48\"]  
47      standard        [24\" x 60\"]  
53      standard         [9\" x 10\"]  
...          ...                  ...  
56956   standard        [22\" x 35\"]  
56957   standard        [22\" x 35\"]  
56958   standard        [22\" x 35\"]  
56959   standard        [22\" x 35\"]  
56960   standard        [22\" x 35\"]  

[7477 rows x 7 columns]

In [12]:
x['matches']=x['matches'].str.get(0)
x['matches']=x['matches'].apply(lambda x:x.lower())
x[['length', 'width']]=x['matches'].str.split(" x ",expand=True,)
x

<ipython-input-12-91d081f8f086>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['matches']=x['matches'].str.get(0)
<ipython-input-12-91d081f8f086>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['matches']=x['matches'].apply(lambda x:x.lower())
C:\Users\groupby\anaconda3\lib\site-packages\pandas\core\frame.py:3065: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: ht

attribute       value    customers_name external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    60715197   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550184   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550603   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63561005   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution            matches    length   width  
44         rules   35-inch x 9-inch   35-inch  9-inch  
45      standard          5\" x 8\"       5\"     8\"  
46      standard        30\" x 48\"      30\"    48\"  
47      standard        24\" x 60\"      24\"    60\"  
53      standard         9\" x 10\"       9\"    10\"  
...          ...                ...       ...     ...  
56956   standard        22\" x 35\"      22\"    35\"  
56957   standard        22\" x 35\"      22\"    35\"  
56958   standard        22\" x 35\"      22\"    35\"  
56959   standard        22\" x 35\"      22\"    35\"  
56960   standard        22\" x 35\"      22\"    35\"  

[7477 rows x 9 columns]

In [13]:
x['length']=x['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\"','',str(measure)))
x['width']=x['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\"','',str(measure)))
x['length']=x['length'].replace('"', '').apply(lambda quote: quote.replace('"', ''))
x['width']=x['width'].apply(lambda quote: quote.replace('"', ''))
x['length']=x['length'].replace('"', '').apply(lambda quote: quote.replace('-',''))
x['width']=x['width'].apply(lambda quote: quote.replace('-',''))
x

<ipython-input-13-0100a92c13c1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['length']=x['length'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-\"','',str(measure)))
<ipython-input-13-0100a92c13c1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['width']=x['width'].apply(lambda measure: re.sub(r'[a-z]|[A-z]|\s|\w*\.\w*\.|\-\"','',str(measure)))
<ipython-input-13-0100a92c13c1>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[

attribute       value    customers_name external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    60715197   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550184   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550603   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63561005   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution            matches length width  
44         rules   35-inch x 9-inch     35     9  
45      standard          5\" x 8\"      5     8  
46      standard        30\" x 48\"     30    48  
47      standard        24\" x 60\"     24    60  
53      standard         9\" x 10\"      9    10  
...          ...                ...    ...   ...  
56956   standard        22\" x 35\"     22    35  
56957   standard        22\" x 35\"     22    35  
56958   standard        22\" x 35\"     22    35  
56959   standard        22\" x 35\"     22    35  
56960   standard        22\" x 35\"     22    35  

[7477 rows x 9 columns]

In [14]:
x.dtypes

attribute         object
value             object
customers_name    object
external_id       object
long_desc         object
resolution        object
matches           object
length            object
width             object
dtype: object

In [15]:
x['length']=x['length'].astype(int)

<ipython-input-15-51b26558676c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['length']=x['length'].astype(int)


In [16]:
x['width']=x['width'].astype(int)
x

<ipython-input-16-06a482d1f1e6>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['width']=x['width'].astype(int)


attribute       value    customers_name external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    60715197   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550184   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550603   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63561005   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution            matches  length  width  
44         rules   35-inch x 9-inch      35      9  
45      standard          5\" x 8\"       5      8  
46      standard        30\" x 48\"      30     48  
47      standard        24\" x 60\"      24     60  
53      standard         9\" x 10\"       9     10  
...          ...                ...     ...    ...  
56956   standard        22\" x 35\"      22     35  
56957   standard        22\" x 35\"      22     35  
56958   standard        22\" x 35\"      22     35  
56959   standard        22\" x 35\"      22     35  
56960   standard        22\" x 35\"      22     35  

[7477 rows x 9 columns]

In [17]:
x['Max'] = x[['length', 'width']].values.max(1)
x['Min'] = x[['length', 'width']].values.min(1)
x

<ipython-input-17-1d2e2fecdef1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Max'] = x[['length', 'width']].values.max(1)
<ipython-input-17-1d2e2fecdef1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x['Min'] = x[['length', 'width']].values.min(1)


attribute       value    customers_name external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    60715197   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550184   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550603   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63561005   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution            matches  length  width  Max  Min  
44         rules   35-inch x 9-inch      35      9   35    9  
45      standard          5\" x 8\"       5      8    8    5  
46      standard        30\" x 48\"      30     48   48   30  
47      standard        24\" x 60\"      24     60   60   24  
53      standard         9\" x 10\"       9     10   10    9  
...          ...                ...     ...    ...  ...  ...  
56956   standard        22\" x 35\"      22     35   35   22  
56957   standard        22\" x 35\"      22     35   35   22  
56958   standard        22\" x 35\"      22     35   35   22  
56959   standard        22\" x 35\"      22     35   35   22  
56960   standard        22\" x 35\"      22     35   35   22  

[7477 rows x 11 columns]

In [18]:
# x['Max']=x['Max'].astype(str)
# x['Final']=x['Max'].apply(lambda x: kwp.extract_keywords(x))
# x

In [19]:
matches=x
matches

attribute       value    customers_name external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    60715197   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    61327016   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550184   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63550603   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond    63561005   
...                        ...         ...               ...         ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond    69648831   
56957  size_width_rugs_runners         n/a  bedbathandbeyond    69648836   
56958  size_width_rugs_runners         n/a  bedbathandbeyond    69648837   
56959  size_width_rugs_runners         n/a  bedbathandbeyond    69649153   
56960  size_width_rugs_runners         n/a  bedbathandbeyond    69649157   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution            matches  length  width  Max  Min  
44         rules   35-inch x 9-inch      35      9   35    9  
45      standard          5\" x 8\"       5      8    8    5  
46      standard        30\" x 48\"      30     48   48   30  
47      standard        24\" x 60\"      24     60   60   24  
53      standard         9\" x 10\"       9     10   10    9  
...          ...                ...     ...    ...  ...  ...  
56956   standard        22\" x 35\"      22     35   35   22  
56957   standard        22\" x 35\"      22     35   35   22  
56958   standard        22\" x 35\"      22     35   35   22  
56959   standard        22\" x 35\"      22     35   35   22  
56960   standard        22\" x 35\"      22     35   35   22  

[7477 rows x 11 columns]

In [20]:
# string = range_string
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [21]:
string = range_string
lst = string.split('\n')
builder_base = f'''
def range_app(num):
'''
for val in lst:
    if val.find(' - ') >-1:
        original_val = val
        lower, upper = val.split(' - ')
        inner_f = f'''
    if num >= {lower} and num <= {upper}:
        return f'{lower}" - {upper}"{metric}'
        '''
        builder_base = builder_base + inner_f
    if val.find('Greater than ') >-1:
        original_val = val
        upper = val.replace('Greater than ','')
        inner_f = f'''
    if num >= {upper}:
        return f'{original_val}"{metric}'
        '''
        builder_base = builder_base + inner_f
final_else = '''
    else:
        return r"n/a"
'''
exec(builder_base + final_else)
print( builder_base + final_else)


def range_app(num):

    if num >=  108:
        return f' Greater than 108"'
        
    if num >=  12 and num <= 23.9:
        return f' 12" - 23.9"'
        
    if num >=  24 and num <= 35.9:
        return f' 24" - 35.9"'
        
    if num >=  36 and num <= 47.9:
        return f' 36" - 47.9"'
        
    if num >=  48 and num <= 59.9:
        return f' 48" - 59.9"'
        
    if num >=  60 and num <= 71.9:
        return f' 60" - 71.9"'
        
    if num >=  72 and num <= 83.9:
        return f' 72" - 83.9"'
        
    if num >=  84 and num <= 95.9:
        return f' 84" - 95.9"'
        
    if num >=  96 and num <= 107.9:
        return f' 96" - 107.9"'
        
    if num >=  0 and num <= 11.9:
        return f' 0" - 11.9"'
        
    else:
        return r"n/a"



In [22]:
# matches['Min']=matches['Min'].astype(int)
# matches['final']=matches['Max'].apply(lambda x: kwp.extract_keywords(x))
# matches.dtypes

In [23]:
matches[curation_col]=matches['Min'].apply(range_app)
matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

<ipython-input-23-3d6147e3b80d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches[curation_col]=matches['Min'].apply(range_app)
<ipython-input-23-3d6147e3b80d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['external_id']=matches['external_id'].apply('="{}"'.format)


attribute       value    customers_name  external_id  \
44     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond  ="60715197"   
45     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond  ="61327016"   
46     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63550184"   
47     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63550603"   
53     size_width_rugs_runners  0" - 11.9"  bedbathandbeyond  ="63561005"   
...                        ...         ...               ...          ...   
56956  size_width_rugs_runners         n/a  bedbathandbeyond  ="69648831"   
56957  size_width_rugs_runners         n/a  bedbathandbeyond  ="69648836"   
56958  size_width_rugs_runners         n/a  bedbathandbeyond  ="69648837"   
56959  size_width_rugs_runners         n/a  bedbathandbeyond  ="69649153"   
56960  size_width_rugs_runners         n/a  bedbathandbeyond  ="69649157"   

                                                                                  long_desc  \
44                                    Mohawk Home® 35-Inch x 9-Inch Mosaic Tile Stair Tread   
45                                     Capel Rugs® Gava Stripes 5\" x 8\" Area Rug in Beige   
46                         Wamsutta® Ultra Fine Reversible 30\" x 48\" Bath Rug in Charcoal   
47                           Wamsutta® Ultra Fine Reversible 24\" x 60\" Bath Rug in Canvas   
53     Nourison Sun and Shade 7'9\" x 10\"10\" Indoor/Outdoor Power-Loomed Area Rug in Blue   
...                                                                                     ...   
56956              EverGrace® 22\" x 35\" The Chef Is Always Right Anti-Fatigue Kitchen Mat   
56957                        EverGrace® 22\" x 35\" Chef's Special Anti-Fatigue Kitchen Mat   
56958                         EverGrace® 22\" x 35\" Kiss The Cook Anti-Fatigue Kitchen Mat   
56959                     Farm Stand Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   
56960                       The Farm Rooster 22\" x 35\" Anti-Fatigue Kitchen Mat in Walnut   

      resolution            matches  length  width  Max  Min  \
44         rules   35-inch x 9-inch      35      9   35    9   
45      standard          5\" x 8\"       5      8    8    5   
46      standard        30\" x 48\"      30     48   48   30   
47      standard        24\" x 60\"      24     60   60   24   
53      standard         9\" x 10\"       9     10   10    9   
...          ...                ...     ...    ...  ...  ...   
56956   standard        22\" x 35\"      22     35   35   22   
56957   standard        22\" x 35\"      22     35   35   22   
56958   standard        22\" x 35\"      22     35   35   22   
56959   standard        22\" x 35\"      22     35   35   22   
56960   standard        22\" x 35\"      22     35   35   22   

      Q:size_width_rugs_runners  
44                   0" - 11.9"  
45                   0" - 11.9"  
46                  24" - 35.9"  
47                  24" - 35.9"  
53                   0" - 11.9"  
...                         ...  
56956               12" - 23.9"  
56957               12" - 23.9"  
56958               12" - 23.9"  
56959               12" - 23.9"  
56960               12" - 23.9"  

[7477 rows x 12 columns]

In [24]:
matches[matches['Q:size_width_rugs_runners']=='n/a']

Empty DataFrame
Columns: [attribute, value, customers_name, external_id, long_desc, resolution, matches, length, width, Max, Min, Q:size_width_rugs_runners]
Index: []

In [25]:
# error----Switch Max and Min when doing length and width

In [26]:
gc = gcloud.Gsheets()

In [27]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

succesfully saved to worksheet:`ul` 
 url = `https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0`


# send to the folder for upload

In [28]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na {attribute} Shawn.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match {attribute} Shawn.csv',index=False)

In [29]:
looks_good('Bed Bath & Beyond', attribute, df, matches)

<ipython-input-28-f8dda3983948>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  non_matches[curation_col] = r'n/a'
